In [37]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,Annotated
from langchain_core.messages import HumanMessage,BaseMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

from langgraph.checkpoint.memory import MemorySaver


In [18]:
load_dotenv()

True

In [19]:
from langgraph.graph.message import add_messages
class ChatState(TypedDict):
    messages:Annotated[list[BaseMessage],add_messages]

In [20]:
llm =ChatOpenAI(
    model='deepseek/deepseek-chat-v3-0324:free',
    api_key=os.environ['OPENROUTER_API_KEY'],
    base_url='https://openrouter.ai/api/v1',
    temperature=0.7
)

In [21]:

def chat_node(state:ChatState):
    #take user query from state
    messages=state['messages']

    #send to LLM
    response=llm.invoke(messages)

    #store back in state and return 
    return {'messages':[response]}

In [38]:
checkpointer=MemorySaver()
graph=StateGraph(ChatState)

#add nodes
graph.add_node('chat_node',chat_node)

#add edges
graph.add_edge(START,'chat_node')
graph.add_edge('chat_node',END)

chatbot=graph.compile(checkpointer=checkpointer)

In [27]:
initial_state={
    'messages':[HumanMessage(content='What is the capital of india?')]
}

In [29]:
chatbot.invoke(initial_state)['messages'][-1].content

"The capital of India is **New Delhi**. It is a part of the larger **National Capital Territory (NCT) of Delhi** and serves as the political and administrative center of the country. \n\n- **Key Facts**:\n  - Houses important government buildings like the **Rashtrapati Bhavan** (President's residence), **Parliament of India**, and **Supreme Court**.\n  - One of the most populous cities in India, blending modernity with rich history (e.g., monuments like the **Red Fort** and **India Gate**).\n  - Established as the capital in **1911** during British rule, replacing Kolkata (then Calcutta).\n\nLet me know if you'd like more details!"

In [ ]:
thread_id='1'
while True:
    user_message=input("Type Here:")
    print('User: 'user_message)

    if user_message.strip().lower() in ['exit','bye','quit']:
        break
    config={'configurable':{'thread_id':thread_id}}
    response=chatbot.invoke({'messages':[HumanMessage(content=user_message)]},config=config)
    print('AI:',response['messages'][-1].content)


hey my name is krish
AI: Hey Krish! 👋 How's it going? What's on your mind today? 😊
what is my name
AI: Your name is **Krish**! (Unless you’re testing me… or secretly a shape-shifting AI in disguise. 🤖👀)  

What’s up? Anything fun or interesting happening today? 😄

AI:             😄 *Gasp* Did you just pull a classic *"Wait, what's my name again?"* test on me? Nice try, Krish—but I’ve got a **steel-trap memory** for introductions (or at least a 0.0001-second cache).  

**Pro tip:** If you *actually* forget your name someday, just check your ID… or yell it in a Starbucks until someone writes it on a cup. ☕✨  

Sooo… what’s the next mission? Philosophy? Memes? Life advice? 🔥
bye
